In [1]:
%load_ext autoreload
%autoreload 1
import ROOT
import JupyROOT.helpers.utils
JupyROOT.helpers.utils.GetGeometryDrawer = lambda:None
import cppyy
from DataFormats.FWLite import Handle
import hist
import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.style.CMS)
import pandas as pd
import sys
sys.path.append("..")
#import edmReader.pythonizations
%aimport edmReader.edmReader
from edmReader.edmReader import EdmReader, MultiEdmReader

Welcome to JupyROOT 6.26/11


In [2]:
from edmReader.edmReader import *

In [3]:
events = EdmReader("/grid_mnt/data_cms_upgrade/cuisset/supercls/alessandro_electrons/supercls-v14-dnn/step3_DNN_1.root")

In [9]:
events.to(0)

True

In [ ]:
events.ticlSuperclusterLinks

std::vector<vector<unsigned int> >[std::vector<unsigned int>[1812,1483],std::vector<unsigned int>[13],std::vector<unsigned int>[90],std::vector<unsigned int>[223],std::vector<unsigned int>[296],std::vector<unsigned int>[317],std::vector<unsigned int>[386],std::vector<unsigned int>[422],std::vector<unsigned int>[439],std::vector<unsigned int>[442],std::vector<unsigned int>[444],std::vector<unsigned int>[453],std::vector<unsigned int>[510],std::vector<unsigned int>[540],std::vector<unsigned int>[543],std::vector<unsigned int>[545],std::vector<unsigned int>[582],std::vector<unsigned int>[631],std::vector<unsigned int>[820],std::vector<unsigned int>[857]]

In [18]:
events.eventAuxiliary().event()

5

In [8]:
links = [[x for x in y] for y in events.ticlSuperclusterLinks]
links

[[1812, 1483],
 [13],
 [90],
 [223],
 [296],
 [317],
 [386],
 [422],
 [439],
 [442],
 [444],
 [453],
 [510],
 [540],
 [543],
 [545],
 [582],
 [631],
 [820],
 [857],
 [1214],
 [1229],
 [1253],
 [1264],
 [1404],
 [1455],
 [1467],
 [1477],
 [1671],
 [1677],
 [1685],
 [1720],
 [1851]]

In [19]:
event_nbs = []
supercls_ids = []
ts_in_supercls_ids = []
ts_ids = []
for evt in events:
    event_nb = evt.eventAuxiliary().event()
    for supercls_id, tsInSupercls_list in enumerate(events.ticlSuperclusterLinks):
        for ts_in_supercls_id, ts_id in enumerate(tsInSupercls_list):
            event_nbs.append(event_nb)
            supercls_ids.append(supercls_id)
            ts_in_supercls_ids.append(ts_in_supercls_id)
            ts_ids.append(ts_id)


In [22]:
df = pd.DataFrame(data=dict(event=event_nbs, supercls_ids=supercls_ids, ts_in_supercls_ids=ts_in_supercls_ids, ts_ids=ts_ids))
df

,event,supercls_ids,ts_in_supercls_ids,ts_ids
0,5,0,0,1812
1,5,0,1,1483
2,5,1,0,13
3,5,2,0,90
4,5,3,0,223
...,...,...,...,...
4799,100,45,0,2009
4800,100,46,0,2034
4801,100,47,0,2400
4802,100,48,0,2589


In [16]:
maker = SuperclsDataframeMaker()
events.toBegin()
for evt in events:
    maker.processEvent(evt, 1)
maker.makeDf()

,ntuple,event_,supercls_id,ts_in_supercls_id,ts_id
0,1,5,0,0,1812
1,1,5,0,1,1483
2,1,5,1,0,13
3,1,5,2,0,90
4,1,5,3,0,223
...,...,...,...,...,...
4799,1,100,45,0,2009
4800,1,100,46,0,2034
4801,1,100,47,0,2400
4802,1,100,48,0,2589


In [17]:
maker.makeDf().to_pickle("/grid_mnt/data_cms_upgrade/cuisset/supercls/alessandro_electrons/supercls-v14-dnn/dataframes/supercls_1_test.pkl.gz")